In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData

### Hyper parameters setting

In [2]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cpu',
    'attrs': ['title'],
    'k': 0,
    'validate':False
}
# torch.cuda.set_device(hparams['device'])

### Construct necessary dictionaries
- already done and the results are in `data/dictionaries`

In [ ]:
# constructBasicDict(attrs=['title'],path='/home/peitian_zhang/Data/MIND')

### View data

In [3]:
vocab, loaders = prepare(hparams, pin_memory=False)

# for encoding news only
# vocab, loaders = prepare(hparams, news=True)

[2021-03-21 09:27:33,701] INFO (root) Hyper Parameters are
{'npratio': 4, 'mode': 'train', 'scale': 'demo', 'batch_size': 10, 'his_size': 50, 'title_size': 15, 'device': 'cpu', 'attrs': ['title'], 'k': 0, 'validate': False}
[2021-03-21 09:27:33,702] INFO (root) preparing dataset...
[2021-03-21 09:27:36,031] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
# loader_train
a = next(iter(loaders[0]))
# loader_dev
b = next(iter(loaders[1]))

In [12]:
embedding = torch.nn.Embedding(20000,10)
embedding(a['user_index']).shape
a['user_index'] = torch.tensor([[1],[2],[3]])
embedding.weight, embedding(a['user_index'])

(Parameter containing:
 tensor([[ 8.6673e-01, -2.3187e+00,  4.8320e-01,  ...,  1.0779e+00,
           7.0819e-01,  5.2822e-01],
         [ 5.1428e-02, -2.5159e-04, -1.0456e+00,  ..., -5.9466e-01,
           2.5224e-01, -1.3118e+00],
         [ 1.1149e+00, -8.8765e-01, -1.4817e-01,  ...,  2.2986e-01,
          -2.6257e-01,  3.2976e-01],
         ...,
         [-1.3801e-01,  1.5101e+00,  6.2164e-01,  ...,  1.3304e+00,
           5.8683e-01,  5.8839e-01],
         [ 7.5770e-01,  2.8950e-01, -2.6903e-01,  ...,  5.4247e-01,
          -4.6608e-01,  2.8433e-01],
         [ 2.8993e-01,  1.8119e+00, -1.8658e-01,  ...,  3.1166e-01,
          -2.9683e+00, -6.0720e-01]], requires_grad=True),
 tensor([[[ 5.1428e-02, -2.5159e-04, -1.0456e+00,  9.7266e-01,  9.2905e-01,
            6.8950e-01, -1.3658e+00, -5.9466e-01,  2.5224e-01, -1.3118e+00]],
 
         [[ 1.1149e+00, -8.8765e-01, -1.4817e-01, -2.8997e-01, -9.8578e-01,
           -4.0253e-01,  1.2189e+00,  2.2986e-01, -2.6257e-01,  3.2976e-01]],
 

### Tailor Data to demo size

In [ ]:
# tailor 2000 impressions from MINDsmall_train to form MINDdemo_train
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [ ]:
analyse(hparams)

## The rest is developing

In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData
from utils.MIND import MIND
import numpy as np
import re
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, IterableDataset
from utils.utils import newsample, getId2idx, word_tokenize_vocab, getVocab, my_collate, worker_init_fn
from torch.utils.data import DataLoader

In [ ]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:1',
    'attrs': ['title'],
    'news_id':True,
    'k': 0,
    'validate':False,
    'bert':'bert-base-uncased'
}
# torch.cuda.set_device(device)

In [ ]:
mind = MIND(hparams, '/home/peitian_zhang/Data/MIND/MINDdemo_dev/news.tsv', '/home/peitian_zhang/Data/MIND/MINDdemo_dev/behaviors.tsv')

loader_train = DataLoader(mind, batch_size=hparams['batch_size'], pin_memory=True,
                                  num_workers=0, drop_last=False, collate_fn=my_collate, worker_init_fn=worker_init_fn)

In [ ]:
record = next(mind.__iter__())
record

In [ ]:
record['candidate_title'].shape, record['candidate_title_pad'].shape, record['clicked_title'].shape, record['clicked_title_pad'].shape

In [ ]:
next(iter(loader_train))